# Access OCR Output Data

This guide shows how to access OCR's wind-adjusted fire risk output data using [Icechunk](https://icechunk.io/en/latest/), a versioned data format for cloud-native geospatial data.

## Prerequisites

- Python environment with `xarray`, `icechunk`, `duckdb` and `lonboard` installed.

## What you'll learn

- How to open and inspect the fire risk raster dataset
- How to work with specific variables and spatial subsets
- How to open and subset the raster sampled building vector dataset

## Raster / Xarray

###  Import required libraries

In [1]:
import icechunk
import xarray as xr

### Connect to the Icechunk repository

OCR's production fire risk data is stored in an Icechunk repository on S3. We'll connect to version `v0.9.0` of the wind-adjusted fire risk output. For valid versions, check out the [OCR's GitHub release page](https://github.com/carbonplan/ocr/releases).

In [2]:
# Configure S3 storage for the Icechunk repository
version = 'v0.12.0'
storage = icechunk.s3_storage(
    bucket='carbonplan-ocr',
    prefix=f'output/fire-risk/tensor/production/{version}/ocr.icechunk',
    anonymous=True,
)

# Open the repository
repo = icechunk.Repository.open(storage)

# Create a read-only session on the main branch
session = repo.readonly_session('main')

### Open the dataset with xarray


In [3]:
# Open the dataset
ds = xr.open_dataset(session.store, engine='zarr', chunks={})
ds

<xarray.Dataset> Size: 652GB
Dimensions:                     (latitude: 97579, longitude: 208881)
Coordinates:
  * latitude                    (latitude) float64 781kB 22.43 22.43 ... 52.48
  * longitude                   (longitude) float64 2MB -128.4 -128.4 ... -64.05
Data variables:
    USFS_RPS                    (latitude, longitude) float32 82GB dask.array<chunksize=(6000, 4500), meta=np.ndarray>
    burn_probability_2047       (latitude, longitude) float32 82GB dask.array<chunksize=(6000, 4500), meta=np.ndarray>
    burn_probability_usfs_2011  (latitude, longitude) float32 82GB dask.array<chunksize=(6000, 4500), meta=np.ndarray>
    burn_probability_2011       (latitude, longitude) float32 82GB dask.array<chunksize=(6000, 4500), meta=np.ndarray>
    burn_probability_usfs_2047  (latitude, longitude) float32 82GB dask.array<chunksize=(6000, 4500), meta=np.ndarray>
    conditional_risk_usfs       (latitude, longitude) float32 82GB dask.array<chunksize=(6000, 4500), meta=np.ndarray>
    wind_risk_2011              (latitude, longitude) float32 82GB dask.array<chunksize=(6000, 4500), meta=np.ndarray>
    wind_risk_2047              (latitude, longitude) float32 82GB dask.array<chunksize=(6000, 4500), meta=np.ndarray>

### Explore the data variables

The dataset contains wind-adjusted fire risk metrics. Let's examine the available variables:

In [4]:
# List all data variables
print('Data variables:')
for var in ds.data_vars:
    print(f'  - {var}: {ds[var].attrs.get("long_name", "No description")}')

Data variables:
  - USFS_RPS: No description
  - burn_probability_2047: No description
  - burn_probability_usfs_2011: No description
  - burn_probability_2011: No description
  - burn_probability_usfs_2047: No description
  - conditional_risk_usfs: No description
  - wind_risk_2011: No description
  - wind_risk_2047: No description


### Select a spatial subset

Extract data for a specific geographic region using coordinate slicing:

In [5]:
# Example: Select data for California region
california_subset = ds.sel(
    latitude=slice(42, 32),  # Southern to Northern California
    longitude=slice(-125, -114),  # Western to Eastern California
)

california_subset

<xarray.Dataset> Size: 286kB
Dimensions:                     (latitude: 0, longitude: 35715)
Coordinates:
  * latitude                    (latitude) float64 0B 
  * longitude                   (longitude) float64 286kB -125.0 ... -114.0
Data variables:
    USFS_RPS                    (latitude, longitude) float32 0B dask.array<chunksize=(0, 2501), meta=np.ndarray>
    burn_probability_2047       (latitude, longitude) float32 0B dask.array<chunksize=(0, 2501), meta=np.ndarray>
    burn_probability_usfs_2011  (latitude, longitude) float32 0B dask.array<chunksize=(0, 2501), meta=np.ndarray>
    burn_probability_2011       (latitude, longitude) float32 0B dask.array<chunksize=(0, 2501), meta=np.ndarray>
    burn_probability_usfs_2047  (latitude, longitude) float32 0B dask.array<chunksize=(0, 2501), meta=np.ndarray>
    conditional_risk_usfs       (latitude, longitude) float32 0B dask.array<chunksize=(0, 2501), meta=np.ndarray>
    wind_risk_2011              (latitude, longitude) float32 0B dask.array<chunksize=(0, 2501), meta=np.ndarray>
    wind_risk_2047              (latitude, longitude) float32 0B dask.array<chunksize=(0, 2501), meta=np.ndarray>

## Vector building dataset
Next we will open and query the raster sampled building dataset stored in the `geoparquet` format.

###  Import required libraries
Here we'll import duckdb and load the spatial extension. The combination of `duckdb` spatial and `geoparquet` allows us to perform GIS queries that are beyond desktop GIS capabilities. 

In [6]:
import duckdb

duckdb.sql("""INSTALL SPATIAL; LOAD SPATIAL; INSTALL HTTPFS; LOAD HTTPFS""")

dataset_uri = f's3://carbonplan-ocr/output/fire-risk/vector/production/{version}/geoparquet/buildings.parquet/**'

### Examine the dataset
- The SQL `describe` command shows us that the dataset contains raster sampled variables as well as geometry columns and regional identifiers.

In [7]:
duckdb.sql(f"""
DESCRIBE
SELECT
   *
FROM
   read_parquet('{dataset_uri}', hive_partitioning = TRUE)
""")

┌────────────────────────────┬────────────────────────────────────────────────────────────┬─────────┬─────────┬─────────┬─────────┐
│        column_name         │                        column_type                         │  null   │   key   │ default │  extra  │
│          varchar           │                          varchar                           │ varchar │ varchar │ varchar │ varchar │
├────────────────────────────┼────────────────────────────────────────────────────────────┼─────────┼─────────┼─────────┼─────────┤
│ USFS_RPS                   │ FLOAT                                                      │ YES     │ NULL    │ NULL    │ NULL    │
│ wind_risk_2011             │ FLOAT                                                      │ YES     │ NULL    │ NULL    │ NULL    │
│ wind_risk_2047             │ FLOAT                                                      │ YES     │ NULL    │ NULL    │ NULL    │
│ burn_probability_2011      │ FLOAT                                        

### Load the first few rows
Using the SQL `LIMIT` command, we can get just the first few rows of the dataset.

In [8]:
duckdb.sql(f"""
SELECT
   *
FROM
   read_parquet('{dataset_uri}', hive_partitioning = TRUE) LIMIT 5""")

┌─────────────┬────────────────┬────────────────┬───────────────────────┬───────────────────────┬───────────────────────┬────────────────────────────┬────────────────────────────┬─────────────────┬─────────┬────────────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬────────────────────────────────────────────────────────────────────────────────────┬─────────────┬────────────┐
│  USFS_RPS   │ wind_risk_2011 │ wind_risk_2047 │ burn_probability_2011 │ burn_probability_2047 │ conditional_risk_usfs │ burn_probability_usfs_2011 │ burn_probability_usfs_2047 │      GEOID      │  state  │     county     │                                                              geometry                                                              │                                        bbox                                        │ county_fips │ state_fips │
│    float    │     float      │     float      │         fl

### Subset by state and county
This geoparquet is partitioned by [state and county using FIPS codes](https://transition.fcc.gov/oet/info/maps/census/fips/fips.txt). 

Let's select data in LA County in California.


In [9]:
CA_FIPS_CODE = '06'
LA_FIPS_CODE = '037'

### Get a count of the number of records in LA County
Using the `COUNT` SQL syntax, we can get the total number of entires in our query.

In [10]:
duckdb.sql(f"""
SELECT
   COUNT(*) AS LA_building_count
FROM
   read_parquet('{dataset_uri}', hive_partitioning = TRUE)
WHERE
   state_fips = '{CA_FIPS_CODE}'
   AND county_fips = '{LA_FIPS_CODE}'""")

┌───────────────────┐
│ LA_building_count │
│       int64       │
├───────────────────┤
│           3127513 │
└───────────────────┘

### Subset by bounding box
Looks like there are about 3 million building polygons in our query. Let's subset that further.

We can get a [bounding box (bbox) for an area around the Palisades fire](http://bboxfinder.com/#34.026381,-118.761864,34.152972,-118.466263).



In [11]:
palisades_bbox = (-118.761864, 34.026381, -118.466263, 34.152972)

palisades_query = duckdb.sql(f"""
SELECT
   wind_risk_2011,
   geometry
FROM
   read_parquet('{dataset_uri}', hive_partitioning = TRUE)
WHERE
   state_fips = '{CA_FIPS_CODE}'
   AND county_fips = '{LA_FIPS_CODE}'
   AND bbox.xmin BETWEEN {palisades_bbox[0]} AND {palisades_bbox[2]}
   AND bbox.ymin BETWEEN {palisades_bbox[1]} AND {palisades_bbox[3]}""")

### Visualize our query
Next we'll use the python plotting library, [lonboard](https://developmentseed.org/lonboard/latest/) to visualize the wind informed risk in an interactive map.


In [12]:
from lonboard import Map, PolygonLayer
from lonboard.colormap import apply_continuous_cmap
from matplotlib import colormaps

layer = PolygonLayer.from_duckdb(
    palisades_query,
    get_fill_color=apply_continuous_cmap(
        palisades_query.fetchdf()['wind_risk_2011'], colormaps['YlOrRd']
    ),
    pickable=True,
)

m = Map(layer, show_tooltip=True)
m